# Lecture 10-3

# Messing around with LLMs with Haystack

Step 0: Get OpenAI API Key

To get an OpenAI API key, you need to sign up for an OpenAI account and subscribe to their API service. 

- Go to the OpenAI developer website. https://platform.openai.com/
- Click on "Sign Up" to create a new account, or "Log In" if you already have an account.
- Click on the Settings Gear. 
- Navigate to "API Keys" 
- You may need to set up a billing plan. If you stick with gpt 3.5, it is cheap. You can buy $5 of credits, which will be more than enough for experimentation.
- Go to the API keys section of your OpenAI account.
- Click on "Create new secret key" to generate a new API key.
- Copy the generated API key.
- Create a `.env` file in your project directory.
- In the `.env` file, enter:

```
OPENAI_API_KEY=text_of_super_secret_api_key
```

- You won't be able to see the api key again once you close the dialog. If you don't store it and lose it, you'll need to create a new API key.  

Step 1: Create a New Virtual Environment

- Open your terminal
- Create a new virtual environment and activate it:

```
python -m venv haystackai
haystackai\Scripts\activate
```

just remember to deactivate your virtual environment at the end

```
deactivate
```


See: https://haystack.deepset.ai/tutorials/27_first_rag_pipeline

Once you've activated the haystack environment, install Jupyter, Haystack, and other necessary packages in your Conda environment:

```
pip install openai python-dotenv
pip install torch
pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=2.2.0"
```

Open the notebook

In [1]:
# cell 1: let's haystack know you are running a tutorial
from haystack.telemetry import tutorial_running

tutorial_running(27)


ModuleNotFoundError: No module named 'haystack'

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()


In [ ]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]


In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()


In [ ]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])


In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)


In [ ]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-3.5-turbo")


In [ ]:
from haystack import Pipeline

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")


In [ ]:
question = "What does Rhodes Statue look like?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


In [ ]:
question = "When did construction for the Rhodes statue begin?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])


In [ ]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What is UCLA?"
]


In [ ]:
for question in examples:
    print(question)
    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})
    print(response["llm"]["replies"][0])


In [26]:
from chatlas import ChatAnthropic
from dotenv import load_dotenv

load_dotenv()

chat = ChatAnthropic(
    model = "claude-3-5-sonnet-20241022"
)
chat.app()


INFO:     Started server process [60062]
INFO:     Waiting for application startup.


In [27]:
class Property:
    def __init__(self, name, price, rent, color):
        self.name = name
        self.price = price
        self.rent = rent
        self.color = color
        self.owner = None
        self.houses = 0

class Player:
    def __init__(self, name):
        self.name = name
        self.position = 0
        self.money = 1500
        self.properties = []
        self.is_in_jail = False

class Monopoly:
    def __init__(self):
        self.players = []
        self.board = []
        self.current_player = 0
        self.initialize_board()
    
    def initialize_board(self):
        # Add properties to the board
        # This is a simplified version
        self.board = [
            Property("Go", 0, 0, None),
            Property("Mediterranean Avenue", 60, 2, "brown"),
            Property("Baltic Avenue", 60, 4, "brown"),
            # ... Add more properties
        ]

    def roll_dice(self):
        return random.randint(1, 6) + random.randint(1, 6)

    def move_player(self, player, steps):
        player.position = (player.position + steps) % 40  # 40 spaces on board

    def play_turn(self, player):
        # Roll dice and move
        steps = self.roll_dice()
        self.move_player(player, steps)
        
        # Get current property
        current_property = self.board[player.position]
        
        # Handle property actions
        if current_property.owner is None and current_property.price > 0:
            # Option to buy property
            if player.money >= current_property.price:
                # Add buy logic here
                pass
        elif current_property.owner is not None:
            # Pay rent
            if current_property.owner != player:
                player.money -= current_property.rent
                current_property.owner.money += current_property.rent

In [28]:
import random

def main():
    game = Monopoly()
    
    # Add players
    game.players.append(Player("Player 1"))
    game.players.append(Player("Player 2"))
    
    # Main game loop
    while True:
        current_player = game.players[game.current_player]
        print(f"\n{current_player.name}'s turn")
        print(f"Current position: {game.board[current_player.position].name}")
        print(f"Current money: ${current_player.money}")
        
        game.play_turn(current_player)
        
        # Move to next player
        game.current_player = (game.current_player + 1) % len(game.players)

if __name__ == "__main__":
    main()


Player 1's turn
Current position: Go
Current money: $1500


IndexError: list index out of range